# Tutorial 2 for R

## Make a scenario of Dantzig's Transport Problem using the *ix modeling platform* (ixmp)

<img style="float: right; height: 80px;" src="_static/R_logo.png">

### Aim and scope of the tutorial

This tutorial uses teh transport problem scenario developed in the first tutorial and illustrates how the ``ixmp`` framework can be applied for scenario analysis in the sense often used in economic or environmental modeling: develop a baseline, create a clone from the baseline with altered parameters or assumptions, and solve the new model. Then, compare the results from the original and altered scenario versions.

In particular, this tutorial will take you through the following steps:
0. Launch an ``ixmp.Platform`` instance and retrieve the ``ixmp.Scenario`` instance of Dantzig's transport problem
0. Retrieve some data from the scenario for illustration of filters
0. Make a clone of the baseline scenario, then check out the clone and make changes:  
   in this case, add a new demand location and transport costs to that city
0. Solve the new scenario
0. Display the solution of both the baseline and the new scenario

### Launching the platform, loading and cloning an existing datastructure

This datastructure will be used to clone an existing scenario and, after varying some input parameters, to solve again the Dantzig's transport problem as well as the solution after solving it in GAMS.

In [ ]:
# Load reticulate, used to access the Python API from R
library(reticulate)

# Import ixmp and message_ix, just as in Python
ixmp <- import("ixmp")


In [ ]:
# launch the ix modeling platform using the local default database
mp <- ixmp$Platform()


In [ ]:
scen_list <- mp$scenario_list()
scen_list

# TODO: the conversion of the Java output of the `scenario_list()` function to a clean R dataframe is not yet implemented


In [ ]:
# details for loading an existing datastructure from the IX modeling platform
model <- "transport problem"
scenario <- "standard"


If you have just run the first, ``R_transport`` tutorial, the existing scenario should appear, and we can load it.
Uncomment and run the following line.

In [ ]:
# load the default version scenario from the first tutorial
# scen <- ixmp$Scenario(mp, model, scenario)


If not (e.g. starting with this tutorial), we can use a function that creates the scenario from scratch in one step:

In [ ]:
ixmp_testing <- import("ixmp.testing")
scen <- ixmp_testing$make_dantzig(mp, solve = ".")


### Retrieve some data from the scenario for illustration of filters

Before cloning a scenario and editing data, this section illustrates two-and-a-half methods to retrieve data for a parameter from a scenario.

In [ ]:
# load the distance parameter
d <- scen$par("d")
d


In [ ]:
# show only the distances for connections from Seattle
d[d["i"] == "seattle", ]


In [ ]:
# for faster access or more complex filtering,
# it may be easier to only load specific parameter elements using a dictionary
ele_filter <- {}
ele_filter$i <- c("seattle")
ele_filter$j <- c("chicago", "topeka")

d_filtered <- scen$par("d", ele_filter)
d_filtered


### Make a clone of the baseline scenario, then check out the clone and edit the scenario

For illustration of a scenario analysis workflow, we add a new demand location ``detroit`` and add a demand level and transport costs to that city.
Because the production capacity does not allow much slack for increased production, we also reduce the demand level in ``chicago``.

In [ ]:
# create a new scenario by cloning the datastructure (without keeping the solution)
scen_detroit <- scen$clone(model, "detroit", annotation = "extend the Transport problem by a new city", keep_solution = FALSE)


In [ ]:
# check out the datastructure to make changes
scen_detroit$check_out()


In [ ]:
# reduce demand
scen_detroit$add_par("b", "chicago", 200, "cases")

# add a new city with demand and distances
scen_detroit$add_set("j", "detroit")
scen_detroit$add_par("b", "detroit", 150, "cases")

d_add <- data.frame(i = c("seattle", "san-diego"), j = c("detroit", "detroit"), value = c(1.7, 1.9), unit = "cases")
scen_detroit$add_par("d", d_add)


In [ ]:
d_add <- data.frame(i = c("seattle", "san-diego"), j = c("detroit", "detroit"), value = c(1.7, 1.9), unit = "cases")
scen_detroit$add_par("d", d_add)


In [ ]:
# save changes to database
comment <- "add new city 'detroit' with demand, reduce demand in 'chicago'"
scen_detroit$commit(comment)
scen_detroit$set_as_default()


### Solve the new scenario

In [ ]:
scen_detroit$solve(model = "dantzig")


### Display and analyze the results

For comparison between the baseline scenario, i.e., the original transport problem, and the "detroit" scenario, we show the solution for both cases.

In [ ]:
# display the objective value of the solution in the baseline scenario
scen$var("z")


In [ ]:
# display the objective value of the solution in the "detroit" scenario
scen_detroit$var("z")


In [ ]:
# display the quantities transported from canning plants to demand locations in the baseline scenario
scen$var("x")


In [ ]:
# display the quantities transported from canning plants to demand locations in the "detroit" scenario
scen_detroit$var("x")


In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints in the baseline scenario
scen$equ("demand")


In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints in the "detroit" scenario
scen_detroit$equ("demand")


### Close the database connection of the ix modeling platform

In [ ]:
# close the connection of the platform instance to the local ixmp database files
mp$close_db()
